<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we have learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Let's first load required libraries:


In [ ]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object


In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing


Let’s see how many of each class is in our data set


In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection


Let's plot some columns to underestand data better:


In [ ]:

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Let's look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

We see that people who get the loan at the end of the week don't pay it off, so let's use Feature binarization to set a threshold value less than day 4


In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values


Let's look at gender:


In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Let's convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding

#### How about education?


In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Features before One Hot Encoding


In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

### Feature Selection


Let's define feature sets, X:


In [ ]:
X = Feature
X[0:5]

What are our lables?


In [ ]:
df['loan_status'].replace(to_replace=['COLLECTION', 'PAIDOFF'], value=[0, 1], inplace=True)
y = df['loan_status'].values
y[0:5]

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split)


In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

*   K Nearest Neighbor(KNN)
*   Decision Tree
*   Support Vector Machine
*   Logistic Regression

\__ Notice:\__

*   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
*   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
*   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.\
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
skf = StratifiedKFold(10)
i = 1

for train, test in skf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    clf = KNeighborsClassifier(n_neighbors = i).fit(X_train,y_train)
    yhat= clf.predict(X_test)
    
    print(i, accuracy_score(y_test, yhat))
    
    i += 1

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7).fit(X,y)

# Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
skf = StratifiedKFold(10)
i = 1

for train, test in skf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    clf = DecisionTreeClassifier(criterion="entropy", max_depth = i).fit(X_train,y_train)
    yhat= clf.predict(X_test)
    
    print(i, accuracy_score(y_test, yhat))
    
    i += 1

In [ ]:
tree = DecisionTreeClassifier(criterion="entropy", max_depth = 6).fit(X,y)

# Support Vector Machine


In [ ]:
from sklearn.svm import SVC

In [ ]:
skf = StratifiedKFold(10)
i = 1

for train, test in skf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    clf = SVC(C = i).fit(X_train,y_train)
    yhat= clf.predict(X_test)
    
    print(i, accuracy_score(y_test, yhat))
    
    i += 1

In [ ]:
svc = SVC(C = 7).fit(X,y)

# Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
skf = StratifiedKFold(10)
i = 1

for train, test in skf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    clf = LogisticRegression(C = i).fit(X_train,y_train)
    yhat= clf.predict(X_test)
    
    print(i, accuracy_score(y_test, yhat))
    
    i += 1

In [ ]:
logr = LogisticRegression(C = 10).fit(X,y)

# Model Evaluation using Test set


In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:


In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation


In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x > 3) else 0)
test_df['Gender'].replace(to_replace=['male', 'female'], value=[0, 1], inplace=True)
test_df['loan_status'].replace(to_replace=['COLLECTION', 'PAIDOFF'], value=[0, 1], inplace=True)

Feature = test_df[['Principal', 'terms', 'age', 'Gender', 'weekend']]
Feature = pd.concat([Feature, pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis=1, inplace=True)

X_test = preprocessing.StandardScaler().fit(Feature).transform(Feature)

y_test = test_df['loan_status'].values

In [ ]:
y_knn = knn.predict(X_test)
y_tree = tree.predict(X_test)
y_svc = svc.predict(X_test)
y_logr = logr.predict(X_test)
y_prob = logr.predict_proba(X_test)

data = {
    "KNN":
        [jaccard_score(y_test, y_knn),
         f1_score(y_test, y_knn),
         np.nan],
    "Decision Tree":
        [jaccard_score(y_test, y_tree),
         f1_score(y_test, y_tree),
         np.nan],
    "SVM":
        [jaccard_score(y_test, y_svc),
         f1_score(y_test, y_svc),
         np.nan],
    "LogisticRegression":
        [jaccard_score(y_test, y_logr),
         f1_score(y_test, y_logr),
         log_loss(y_test, y_prob)]
}

# dataframe from dict
results = pd.DataFrame.from_dict(data, orient='index', columns=['Jaccard',
                                                                'F1-score',
                                                                'LogLoss'])
results.index.name = "Algorithm"

In [ ]:
results

# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |
